## Desafío Lunes Semana 6 - Clasificación desde la econometría
###  Gustavo Morales, G10 - 14.Oct.2019

#### Ejercicio 1

In [1]:
import pandas as pd
import numpy as np
import statsmodels.formula.api as smf
from functools import reduce

In [2]:
df = pd.read_csv("southafricanheart.csv")
df = df.drop(columns='Unnamed: 0')

In [3]:
df.sample()

,sbp,tobacco,ldl,adiposity,famhist,typea,obesity,alcohol,age,chd
359,152,1.68,3.58,25.43,Absent,50,27.03,0.0,32,0


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 462 entries, 0 to 461
Data columns (total 10 columns):
sbp          462 non-null int64
tobacco      462 non-null float64
ldl          462 non-null float64
adiposity    462 non-null float64
famhist      462 non-null object
typea        462 non-null int64
obesity      462 non-null float64
alcohol      462 non-null float64
age          462 non-null int64
chd          462 non-null int64
dtypes: float64(5), int64(4), object(1)
memory usage: 36.2+ KB


#### Ejercicio 2

Binarizando `famhist`:

In [5]:
print(df['famhist'].value_counts())

Absent     270
Present    192
Name: famhist, dtype: int64


In [6]:
df['is_Present'] = np.where(df['famhist']=='Present', 1, 0)

In [7]:
df.sample()

,sbp,tobacco,ldl,adiposity,famhist,typea,obesity,alcohol,age,chd,is_Present
379,154,4.5,4.68,39.97,Absent,61,33.17,1.54,64,1,0


Generando el modelo con `smf.logit()` y ajustándolo con `.fit()`:

In [8]:
mod_logit = smf.logit('chd ~ is_Present', df).fit()

Optimization terminated successfully.
         Current function value: 0.608111
         Iterations 5


Y la función logística estándar:

In [9]:
def inverse_logit(x):
    return 1 / (1 + np.exp(-x))

In [10]:
def estimate(model, x):
    return model.params['is_Present']*x + model.params['Intercept']

In [11]:
p1 = inverse_logit(estimate(mod_logit, 1))
p0 = inverse_logit(estimate(mod_logit, 0))

In [12]:
print(f"La probabilidad de un individuo CON antecedentes familiares de tener una enfermedad coronaria es de un {p1:.2f}.")

La probabilidad de un individuo CON antecedentes familiares de tener una enfermedad coronaria es de un 0.50.


In [13]:
print(f"La probabilidad de un individuo SIN antecedentes familiares de tener una enfermedad coronaria es de un {p0:.2f}.")

La probabilidad de un individuo SIN antecedentes familiares de tener una enfermedad coronaria es de un 0.24.


In [14]:
print(f"La diferencia en la probabilidad entre un individuo CON antecedentes y otro SIN antecedentes es de {p1-p0:.2f}.")

La diferencia en la probabilidad entre un individuo CON antecedentes y otro SIN antecedentes es de 0.26.


Replicando el modelo con `smf.ols()`:

In [15]:
mod_lin = smf.ols('chd ~ is_Present', df).fit()
mod_lin.summary2().tables[1]

,Coef.,Std.Err.,t,P>|t|,[0.025,0.975]
Intercept,0.237037,0.027922,8.489319,2.886599e-16,0.182167,0.291907
is_Present,0.262963,0.043313,6.071289,2.657629e-09,0.177848,0.348078


In [16]:
m = mod_logit.params['is_Present']
b = mod_logit.params['Intercept']

Usando $\beta/4$:

In [17]:
print(m/4, b/4)

0.29224827135747733 -0.2922482713574774


**(R)** Es decir, me dan cifras similares al modelo lineal ajustando con `smf.ols()` (para un Linear Probability Model es lo mismo).

#### Ejercicio 3

Para generar el argumento `formula_like` que se usa como input en los modelos usando `smf`:

In [18]:
def generate_fl(variables, dep_variable):
    string = dep_variable + ' ~ '
    for var in variables:
        if var != dep_variable and var != 'famhist':
            string += var
            string += ' + '
    string = string[:-3]
    return string

In [19]:
generate_fl(df.columns.tolist(), 'chd')

'chd ~ sbp + tobacco + ldl + adiposity + typea + obesity + alcohol + age + is_Present'

In [20]:
mod_logit_sat = smf.logit(generate_fl(df.columns.tolist(), 'chd'), df).fit()

Optimization terminated successfully.
         Current function value: 0.510974
         Iterations 6


In [21]:
mod_logit_sat.summary2()

<class 'statsmodels.iolib.summary2.Summary'>
"""
                         Results: Logit
=================================================================
Model:              Logit            Pseudo R-squared: 0.208     
Dependent Variable: chd              AIC:              492.1400  
Date:               2019-10-15 11:40 BIC:              533.4957  
No. Observations:   462              Log-Likelihood:   -236.07   
Df Model:           9                LL-Null:          -298.05   
Df Residuals:       452              LLR p-value:      2.0548e-22
Converged:          1.0000           Scale:            1.0000    
No. Iterations:     6.0000                                       
------------------------------------------------------------------
               Coef.   Std.Err.     z     P>|z|    [0.025   0.975]
------------------------------------------------------------------
Intercept     -6.1507    1.3083  -4.7015  0.0000  -8.7149  -3.5866
sbp            0.0065    0.0057   1.1350  0.2564  -0.0047   0.0177
tobacco        0.0794    0.0266   2.9838  0.0028   0.0272   0.1315
ldl            0.1739    0.0597   2.9152  0.0036   0.0570   0.2909
adiposity      0.0186    0.0293   0.6346  0.5257  -0.0388   0.0760
typea          0.0396    0.0123   3.2138  0.0013   0.0154   0.0637
obesity       -0.0629    0.0442  -1.4218  0.1551  -0.1496   0.0238
alcohol        0.0001    0.0045   0.0271  0.9784  -0.0087   0.0089
age            0.0452    0.0121   3.7285  0.0002   0.0215   0.0690
is_Present     0.9254    0.2279   4.0605  0.0000   0.4787   1.3720
=================================================================

"""

Una significancia de $95\%$ corresponde a un p-value de $5\%$, es decir

In [22]:
CRIT_VALUE = 5/100

In [23]:
table = mod_logit_sat.summary2().tables[1]['P>|z|']

In [24]:
high_sig = [ table.index[i] for i, value in enumerate(table) if value <= CRIT_VALUE and i != 0 ]

In [25]:
string2 = 'chd ~ '
for var in high_sig:
    if var != 'chd':
        string2 += var
        string2 += ' + '
string2 = string2[:-3]
print(string2)

chd ~ tobacco + ldl + typea + age + is_Present


In [26]:
mod_logit_high_sig = smf.logit(string2, df).fit()

Optimization terminated successfully.
         Current function value: 0.514811
         Iterations 6


In [27]:
mod_logit_high_sig.summary2()

<class 'statsmodels.iolib.summary2.Summary'>
"""
                         Results: Logit
=================================================================
Model:              Logit            Pseudo R-squared: 0.202     
Dependent Variable: chd              AIC:              487.6856  
Date:               2019-10-15 11:40 BIC:              512.4990  
No. Observations:   462              Log-Likelihood:   -237.84   
Df Model:           5                LL-Null:          -298.05   
Df Residuals:       456              LLR p-value:      2.5537e-24
Converged:          1.0000           Scale:            1.0000    
No. Iterations:     6.0000                                       
------------------------------------------------------------------
               Coef.   Std.Err.     z     P>|z|    [0.025   0.975]
------------------------------------------------------------------
Intercept     -6.4464    0.9209  -7.0004  0.0000  -8.2513  -4.6416
tobacco        0.0804    0.0259   3.1057  0.0019   0.0297   0.1311
ldl            0.1620    0.0550   2.9470  0.0032   0.0543   0.2697
typea          0.0371    0.0122   3.0505  0.0023   0.0133   0.0610
age            0.0505    0.0102   4.9442  0.0000   0.0305   0.0705
is_Present     0.9082    0.2258   4.0228  0.0001   0.4657   1.3507
=================================================================

"""

**(R)** Comparando `Log-Likelihood` de cada modelo, puedo decir que el mejor modelo es el que tiene las variables con mayor significancia aunque varíe marginalmente.

**(R)** La presión sanguínea sistólica es la variable que tiene mayor efecto en la presencia de una enfermedad coronaria. En orden decreciente de importancia, las variables que le siguen son promedio de tabaco al día, edad, e índice de personalidad tipo A.

#### Ejercicio 4

In [28]:
def compute_log_probability_ldl(dataframe, model, variables, stat, x):
    value = 0
    for var in variables:
        if var[:3] == 'is_':
            value += model.params[var]*x + model.params[var]*0
        elif var != 'ldl':
            value += model.params[var]*df[var].mean()
        elif var == 'ldl':
            if stat == 'max':
                value += model.params[var]*df[var].max()
            elif stat == 'min':
                value += model.params[var]*df[var].min()
            elif stat == 'mean':
                value += model.params[var]*df[var].mean()
    value += model.params['Intercept']  # log-odds
    return inverse_logit(value)

In [29]:
p_mean_1 = compute_log_probability_ldl(df, mod_logit_high_sig, high_sig, 'mean', 1)
p_mean_0 = compute_log_probability_ldl(df, mod_logit_high_sig, high_sig, 'mean', 0)
p_max_1 = compute_log_probability_ldl(df, mod_logit_high_sig, high_sig, 'max', 1)
p_max_0 = compute_log_probability_ldl(df, mod_logit_high_sig, high_sig, 'max', 0)
p_min_1 = compute_log_probability_ldl(df, mod_logit_high_sig, high_sig, 'min', 1)
p_min_0 = compute_log_probability_ldl(df, mod_logit_high_sig, high_sig, 'min', 0)

In [30]:
print(f"La probabilidad de tener una enfermedad coronaria para un individuo:\n\
 - con caracteristicas similares al promedio de la muestra;\n\
 - con antecedentes de familiares con enfermedades coronarias\n\
es de un {p_mean_1*100:.2f}%")

La probabilidad de tener una enfermedad coronaria para un individuo:
 - con caracteristicas similares al promedio de la muestra;
 - con antecedentes de familiares con enfermedades coronarias
es de un 41.42%


In [31]:
print(f"La probabilidad de tener una enfermedad coronaria para un individuo:\n\
 - con caracteristicas similares al promedio de la muestra;\n\
 - sin antecedentes de familiares con enfermedades coronarias\n\
es de un {p_mean_0*100:.2f}%")

La probabilidad de tener una enfermedad coronaria para un individuo:
 - con caracteristicas similares al promedio de la muestra;
 - sin antecedentes de familiares con enfermedades coronarias
es de un 22.19%


In [32]:
print(f"La probabilidad de tener una enfermedad coronaria para un individuo:\n\
 - con altos niveles de lipoproteína de baja densidad;\n\
 - con antecedentes de familiares con enfermedades coronarias\n\
es de un {p_max_1*100:.2f}%")

La probabilidad de tener una enfermedad coronaria para un individuo:
 - con altos niveles de lipoproteína de baja densidad;
 - con antecedentes de familiares con enfermedades coronarias
es de un 79.72%


In [33]:
print(f"La probabilidad de tener una enfermedad coronaria para un individuo:\n\
 - con altos niveles de lipoproteína de baja densidad;\n\
 - sin antecedentes de familiares con enfermedades coronarias\n\
es de un {p_max_0*100:.2f}%")

La probabilidad de tener una enfermedad coronaria para un individuo:
 - con altos niveles de lipoproteína de baja densidad;
 - sin antecedentes de familiares con enfermedades coronarias
es de un 61.32%


In [34]:
print(f"La probabilidad de tener una enfermedad coronaria para un individuo:\n\
 - con bajos niveles de lipoproteína de baja densidad;\n\
 - con antecedentes de familiares con enfermedades coronarias\n\
es de un {p_min_1*100:.2f}%")

La probabilidad de tener una enfermedad coronaria para un individuo:
 - con bajos niveles de lipoproteína de baja densidad;
 - con antecedentes de familiares con enfermedades coronarias
es de un 27.77%


In [35]:
print(f"La probabilidad de tener una enfermedad coronaria para un individuo:\n\
 - con bajos niveles de lipoproteína de baja densidad;\n\
 - sin antecedentes de familiares con enfermedades coronarias\n\
es de un {p_min_0*100:.2f}%")

La probabilidad de tener una enfermedad coronaria para un individuo:
 - con bajos niveles de lipoproteína de baja densidad;
 - sin antecedentes de familiares con enfermedades coronarias
es de un 13.42%


En resumen:

In [36]:
indices = ['con antecedentes', 'sin antecedentes']
df1 = pd.DataFrame([p_mean_1, p_mean_0], index=indices, columns=['promedio'])*100
df2 = pd.DataFrame([p_max_1, p_max_0], index=indices, columns=['altos niveles'])*100
df3 = pd.DataFrame([p_min_1, p_min_0], index=indices, columns=['bajos niveles'])*100

In [37]:
pd.concat([df1, df2, df3], axis=1)

,promedio,altos niveles,bajos niveles
con antecedentes,41.418659,79.717878,27.771470
sin antecedentes,22.185998,61.315148,13.423737
